In [24]:
import numpy as np
from math import ceil
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import sys
import re
import pandas as pd
import torch
import torch.nn as nn
import scipy
from scipy import ndimage
import torchvision
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from jupyterthemes import jtplot
import random
import cv2
from torchvision import models
from tqdm import tqdm

data = pd.read_csv('../../fer2013/fer2013.csv')

In [25]:
print(len(data))
print(len(data.loc[0, 'pixels'].split(' ')))
print(data.groupby('Usage').count()[['emotions']])

#Frequency of each label
print('0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral')
table = data.groupby('emotions').count()[['Usage']]
table['Pct'] = table['Usage']/table['Usage'].sum()
table['Pct'] = table['Pct'].map(lambda x: round(x, 3)*100)
table

35887
2304
             emotions
Usage                
PrivateTest      3589
PublicTest       3589
Training        28709
0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral


,Usage,Pct
emotions,,
0,4953,13.8
1,547,1.5
2,5121,14.3
3,8989,25.0
4,6077,16.9
5,4002,11.2
6,6198,17.3


## 1) Converting to numpy datasets

In [26]:
def convert_to_numpy(data):
    X = data[:,1]
    X = np.asarray([np.asarray(X[i].split(" ")) for i in range(X.shape[0])])
    X = np.asarray([X[i].reshape(48,48).astype(int) for i in range(X.shape[0])])
    y = data[:,0]
    return (X,y)

In [27]:
train = data[data['Usage'] == 'Training']
train_X, train_Y = convert_to_numpy(train.values)
valid = data[data['Usage'] == 'PrivateTest']
valid_X, valid_Y = convert_to_numpy(valid.values)
test = data[data['Usage'] == 'PublicTest']
test_X, test_Y = convert_to_numpy(test.values)

## 2) Function to extract the batches from the dataset

In [28]:
def data_iter(x, y, batch_size):
    dataset_size = x.shape[0]
    start = -1 * batch_size
    order = list(range(dataset_size))
    random.shuffle(order)

    while True:
        start += batch_size
        if start > dataset_size - batch_size:
            break   
        batch_indices = order[start:start + batch_size]
        yield np.asarray([x[index] for index in batch_indices]) ,np.asarray([y[index] for index in batch_indices])

## 3) Model 

## CNN Implementation

### 1) First implementation (Basic)

In [29]:
class CNN(nn.Module):
    """
    CNN model
    """
       
    def __init__(self, kernel_size, num_labels, n_layers=1, dropout=0.1):
       
        """
        @param vocab_size: size of the vocabulary. 
        @param emb_dim: size of the word embedding
        """
        super(CNN,self).__init__()

        self.conv1 = nn.Sequential(
                            nn.Conv2d(1, 10, kernel_size=kernel_size, stride=1, padding=1),
                            nn.ReLU(),
                            nn.MaxPool2d(kernel_size=2),
                            )
        self.conv2 = nn.Sequential(
                            nn.Conv2d(10, 20, kernel_size=kernel_size, stride=1, padding=1),
                            nn.ReLU(),
                            nn.MaxPool2d(kernel_size=2),
                            )
        self.out = nn.Linear(20*12*12, num_labels)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        x = x.unsqueeze(1) # (N,Ci,W,D)
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1) # Note that normally ppl dont use dropout in CNN
        x = self.out(x)
        return torch.nn.functional.softmax(x)

## 4) Training stage setup

In [30]:
def early_stop(val_acc_history, t=2, required_progress=0.001):    
    cnt = 0 # initialize the count --> to store count of cases where difference in
                                    #  accuracy is less than required progress.
    
    if(len(val_acc_history) > 0): # if list has size > 0 
        for i in range(t): # start the loop
            index = len(val_acc_history) - (i+1) # start from the last term in list and move to the left
            if (index >= 1): # to check if index != 0 --> else we can't compare to previous value
                if (abs(val_acc_history[index] - val_acc_history[index-1]) < required_progress):
                    cnt += 1 # increase the count value
                else:
                    break # break if difference is grea-ter 
    
    if(cnt != t): # if count is equal to t, return True
        return False
    else:
        return True
    

def train(train_X, train_Y, valid_X, valid_Y, optimizer, model, batch_size, num_epochs, criterion, to_Add_Softmax=False, is_inception=False):
    losses = []
    total_batches = int(train_X.shape[0]/ batch_size)
    validation_losses = []
    
    eval_every = 10
    print_every = 10
    validate_every = int((eval_every/100)*total_batches)
    show_every = int((print_every/100)*total_batches)
    
    for epoch in range(1, num_epochs+1):
        stop_training = False
        train_data = data_iter(train_X, train_Y, batch_size)
        for i, (x,y) in enumerate(train_data):
            x = Variable(torch.from_numpy(x).type(torch.FloatTensor))
            y = Variable(torch.from_numpy(y).type(torch.LongTensor))
            model.train(True)
            optimizer.zero_grad()
            outputs = model(x)
            if is_inception == True:
                outputs = outputs[0]
            if to_Add_Softmax == True:
                outputs = nn.functional.softmax(outputs)
            loss = criterion(outputs, y)
            losses.append(loss.data[0])
            loss.backward()


            optimizer.step()
            
            if (i+1)%validate_every == 0:
                valid_loss_temp = []
                valid_data = data_iter(valid_X, valid_Y, batch_size)
                for j, (v_x, v_y) in enumerate(valid_data):
                    v_x = Variable(torch.from_numpy(v_x).type(torch.FloatTensor))
                    v_y = Variable(torch.from_numpy(v_y).type(torch.LongTensor))
                    model.eval()
                    val_outputs = model(v_x)
                    eval_loss = criterion(val_outputs, v_y)
                    valid_loss_temp.append(eval_loss.data[0])
                validation_losses.append(np.mean(valid_loss_temp))
                stop_training = early_stop(validation_losses, 3)
                
            if stop_training:
                print("earily stop triggered")
                break
            if (i+1) % show_every == 0:
                print('Epoch: [{0}/{1}], Step: [{2}/{3}], Train loss: {4}, Validation loss:{5}'.format(
                           epoch, num_epochs, i+1, total_batches, np.mean(losses)/(total_batches*epoch), np.mean(np.array(validation_losses))))
        if stop_training == True:
            break

## 5) Training the model

### 1) first model

In [31]:
def calculate_weights(train_Y, valid_Y, test_Y):
    
    from scipy.stats import itemfreq as freq
    Y = np.concatenate((train_Y, valid_Y, test_Y))
    weight = freq(Y)[:, 1]
    weight = 1/weight*len(Y)
    return weight

In [33]:
num_labels = 7
num_epochs = 5
learning_rate = 0.01
kernel_size = 3
batch_size = 80

model = CNN( kernel_size, num_labels, n_layers=1, dropout=0.1)
weights = calculate_weights(train_Y, valid_Y, test_Y)
weights = torch.FloatTensor(weights)
criterion = nn.NLLLoss(weight=weights)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(train_X, train_Y, valid_X, valid_Y, optimizer, model, batch_size, num_epochs, criterion, False, False)

C:\Users\akash\Anaconda3\lib\site-packages\ipykernel\__main__.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: [1/5], Step: [35/358], Train loss: -0.0019272052042977105, Validation loss:-0.6499483023177494
Epoch: [1/5], Step: [70/358], Train loss: -0.0019362375389930257, Validation loss:-0.6535193544219841
Epoch: [1/5], Step: [105/358], Train loss: -0.0019157088708294044, Validation loss:-0.6530159840529616
Epoch: [1/5], Step: [140/358], Train loss: -0.0019162179282305248, Validation loss:-0.6544163341379978
Epoch: [1/5], Step: [175/358], Train loss: -0.0018970537965809547, Validation loss:-0.6547612467272714
Epoch: [1/5], Step: [210/358], Train loss: -0.0019042137190679756, Validation loss:-0.6550600633138056
Epoch: [1/5], Step: [245/358], Train loss: -0.0019066129340615543, Validation loss:-0.6536110183345033
Epoch: [1/5], Step: [280/358], Train loss: -0.0019117102463984622, Validation loss:-0.6536332568238404
Epoch: [1/5], Step: [315/358], Train loss: -0.0019232594518677566, Validation loss:-0.6525290673504575
Epoch: [1/5], Step: [350/358], Train loss: -0.0019231603993288726, Validati

In [52]:
def get_accuracy_on_subset(model, X, Y, batch_size):
    iter_data = data_iter(X, Y, batch_size)
    total_batches = int(X.shape[0]/ batch_size)
    accuracy_list = []
    pred_output = []
    true_output = []

    for i, (x,y) in enumerate(iter_data):
        x = Variable(torch.from_numpy(x).type(torch.FloatTensor))
        test_output = model(x).type(torch.FloatTensor)
        pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
        accuracy = sum(pred_y == y)/len(y)
        accuracy_list.append(accuracy)
        pred_output.extend(pred_y)
        true_output.extend(y)
    return np.mean(accuracy_list), confusion_matrix(true_output, pred_output)

get_accuracy_on_subset(model, test_X, test_Y, 80)

C:\Users\akash\Anaconda3\lib\site-packages\ipykernel\__main__.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


(0.14687500000000001, array([[  4,  16, 322,   4,   1,  11, 101],
        [  0,   3,  40,   0,   0,   2,  11],
        [  2,  13, 338,   3,   0,  15, 118],
        [  4,  45, 636,   3,   1,  28, 162],
        [  4,  24, 462,   7,   0,  15, 128],
        [  3,  14, 294,   0,   0,   5,  90],
        [  3,  43, 350,   8,   0,  23, 164]], dtype=int64))

### 2) second model

#### 2.1) Zoom the image and convert 1 channel to 3 channel dataset

In [11]:
def to_rgb1a(data, w, h):

    R, _, _ = data.shape
    temp = np.zeros((R, 3, w, h))
    
    for i in tqdm(range(R)):
        im = data[0]
        ret = np.empty((3, w, h), dtype=np.uint8)
        im = cv2.resize(im.astype(float), (w, h), interpolation=cv2.INTER_LINEAR)
        ret[0, :, :] =  ret[1, :, :] =  ret[2, :, :] =  im
        temp[i] = ret
        data = np.delete(data, 0, 0)
    
    return temp

#### Experimenting with a smaller dataset (as running imagenet architectures locally is very time consuming)

In [13]:
old_train_X = train_X
train_X = to_rgb1a(old_train_X[0:1000], 227, 227)
old_valid_X = valid_X
valid_X = to_rgb1a(old_valid_X[0:500], 227, 227)
old_test_X = test_X
test_X = to_rgb1a(old_test_X[0:500], 227, 227)

100%|██████████| 500/500 [00:01<00:00, 485.77it/s]


#### 2.2) Using the pretrained models

In [61]:
### Still some modification needs to be made

alexnet = models.alexnet(pretrained=True)
num_ftrs = alexnet.classifier

## Modifying and training only the last layer
for param in alexnet.parameters():
    param.requires_grad = False

alexnet.classifier._modules['6'] = nn.Linear(4096, num_labels)
# num_ftrs = alexnet.classifier[6].in_features
# alexnet.classifier[6].out_features = num_labels
for param in alexnet.classifier[6].parameters():
    param.requires_grad = True
for param in alexnet.classifier[5].parameters():
    param.requires_grad = True

params = filter(lambda p: p.requires_grad, alexnet.parameters())
    
#optimizer = optim.Adam(filter(lambda p: p.requires_grad, alexnet.parameters()), alexnet.classifier.parameters())
optimizer = optim.Adam(params, lr=0.0001)
num_epochs = 1

train(train_X, train_Y, valid_X, valid_Y, optimizer, alexnet, batch_size, num_epochs, criterion, to_Add_Softmax=True, is_inception=False)

C:\Users\akash\Anaconda3\lib\site-packages\ipykernel\__main__.py:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


KeyboardInterrupt: 

#### 3) resnet

In [66]:
resnet.layer4[2]

Bottleneck(
  (conv1): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
  (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
  (conv3): Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn3): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU(inplace)
)

In [ ]:
resnet = models.resnet50(pretrained=True)
# freeze all model parameters
for param in resnet.parameters():
    param.requires_grad = False

# new final layer with 7 classes
num_ftrs = resnet.fc.in_features
resnet.fc = torch.nn.Linear(num_ftrs, num_labels)

for param in resnet.fc.parameters():
    param.requires_grad = True
for param in resnet.layer4[2].parameters():
    param.requires_grad = True

params = filter(lambda p: p.requires_grad, resnet.parameters())

optimizer = optim.Adam(params, lr=0.0001)
num_epochs = 1

train(train_X, train_Y, valid_X, valid_Y, optimizer, resnet, batch_size, num_epochs, criterion, to_Add_Softmax=True, is_inception=False)

#### 4) Inception

##### Inception requires input size to be (299, 299)

In [14]:
train_X = to_rgb1a(old_train_X[0:1000], 299, 299)
valid_X = to_rgb1a(old_valid_X[0:500], 299, 299)
test_X = to_rgb1a(old_test_X[0:500], 299, 299)

100%|██████████| 500/500 [00:01<00:00, 407.59it/s]


In [ ]:
incptn = models.inception_v3(pretrained=True)
# freeze all model parameters
for param in incptn.parameters():
    param.requires_grad = False

# new final layer with 7 classes
num_ftrs = incptn.fc.in_features
incptn.fc = torch.nn.Linear(num_ftrs, num_labels)

for param in incptn.fc.parameters():
    param.requires_grad = True

for param in incptn.Mixed_7c.parameters():
    param.requires_grad = True

params = filter(lambda p: p.requires_grad, incptn.parameters())
optimizer = optim.Adam(params, lr=0.0001)
num_epochs = 1

train(train_X, train_Y, valid_X, valid_Y, optimizer, incptn, batch_size, num_epochs, criterion, to_Add_Softmax=True, is_inception=True)

## 6) Calculating accuracy on test set

In [30]:
#test_output = model(Variable(torch.from_numpy(test_X).type(torch.FloatTensor)))
resnet.train(False)
test_output = resnet(Variable(torch.from_numpy(test_X).type(torch.FloatTensor)))
pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
accuracy = sum(pred_y == test_Y[0:500])/len(test_Y[0:500])
print(accuracy)

0.156
